Supervisor Process to manage the workers.
---
Diogo Pessoa

Workers in this contexts are other Jupyter notebooks with smaller scope of work.


In [2]:
import pandas as pd

combined_data = pd.read_csv('combined_data.csv').set_index('video_id')
label = combined_data['label']
features = combined_data.drop(['label'], axis=1)